In [ ]:
import numpy as np
import pandas as pd
import torch

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def build_and_train_model(X_train, y_train, X_test, y_test, epochs=200):
    # Простое преобразование в тензоры
    X_train_tensor = torch.FloatTensor(X_train)
    y_train_tensor = torch.FloatTensor(y_train).unsqueeze(1)  
    X_test_tensor = torch.FloatTensor(X_test)
    
    torch.manual_seed(42)
    
    input_size = X_train.shape[1]
    W1 = torch.nn.Parameter(torch.randn(input_size, 64) * 0.1)
    b1 = torch.nn.Parameter(torch.zeros(64))
    W2 = torch.nn.Parameter(torch.randn(64, 32) * 0.1)
    b2 = torch.nn.Parameter(torch.zeros(32))
    W3 = torch.nn.Parameter(torch.randn(32, 1) * 0.1)
    b3 = torch.nn.Parameter(torch.zeros(1))
    
    optimizer = torch.optim.Adam([W1, b1, W2, b2, W3, b3], lr=0.001)
    
    best_loss = float('inf')
    patience = 50
    no_improvement = 0
    
    for epoch in range(epochs):
        layer1 = torch.relu(X_train_tensor @ W1 + b1)
        layer2 = torch.relu(layer1 @ W2 + b2)
        predictions = layer2 @ W3 + b3
        
        loss = torch.mean((predictions - y_train_tensor) ** 2)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if loss.item() < best_loss:
            best_loss = loss.item()
            no_improvement = 0
        else:
            no_improvement += 1
            
        if no_improvement >= patience:
            print(f'Early stopping at epoch {epoch}')
            break
            
        if epoch % 50 == 0:
            print(f'Epoch {epoch}, Loss: {loss.item():.4f}')
    
    with torch.no_grad():
        layer1_test = torch.relu(X_test_tensor @ W1 + b1)
        layer2_test = torch.relu(layer1_test @ W2 + b2)
        y_pred_scaled = (layer2_test @ W3 + b3).numpy().flatten()
    
    return y_pred_scaled

# Основной код
df_paris = pd.read_csv('ParisHousing.csv')

X_paris = df_paris.drop(columns=['price'])
y_paris = df_paris['price']

X_paris_train, X_paris_test, y_paris_train, y_paris_test = train_test_split(
    X_paris, y_paris, test_size=0.2, random_state=42
)

# МАСШТАБИРУЕМ И X И Y!
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_paris_train_scaled = scaler_X.fit_transform(X_paris_train)
X_paris_test_scaled = scaler_X.transform(X_paris_test)

y_paris_train_scaled = scaler_y.fit_transform(y_paris_train.values.reshape(-1, 1)).flatten()
y_paris_test_scaled = scaler_y.transform(y_paris_test.values.reshape(-1, 1)).flatten()

# Обучаем на масштабированных данных
y_pred_scaled = build_and_train_model(
    X_paris_train_scaled, y_paris_train_scaled, 
    X_paris_test_scaled, y_paris_test_scaled,
    epochs=500
)

# Преобразуем предсказания обратно в исходный масштаб
y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1)).flatten()

# Считаем метрики на исходных данных
mae_paris = mean_absolute_error(y_paris_test, y_pred)
mse_paris = mean_squared_error(y_paris_test, y_pred)
r2_paris = r2_score(y_paris_test, y_pred)

print(f"Paris Housing - MAE: {mae_paris:.4f}, MSE: {mse_paris:.4f}, R2: {r2_paris:.4f}")

Epoch 0, Loss: 1.0225
Epoch 50, Loss: 0.1458
Epoch 100, Loss: 0.0054
Epoch 150, Loss: 0.0032
Epoch 200, Loss: 0.0025
Epoch 250, Loss: 0.0021
Epoch 300, Loss: 0.0017
Epoch 350, Loss: 0.0015
Epoch 400, Loss: 0.0013
Epoch 450, Loss: 0.0011
Paris Housing - MAE: 86099.0418, MSE: 11867478096.5818, R2: 0.9986
